# Enter the dataset paths , the percentage for the data split and the parameters to train the model

### Before starting , once refer the readme.md file in the location "/home/viso/sample_dataset"

# If you want to download the predefined dataset follow the next 2 steps or else you can skip that

In [1]:
!python3 tools/misc/download_dataset.py --dataset-name cat --save-dir ./data/cat --unzip --delete

100%|████████████████████████████████████████| 217M/217M [00:22<00:00, 10.2MB/s]
Unzipping cat_dataset.zip
Delete data/cat/cat_dataset.zip


In [2]:
ls data/cat/annotations/

annotations_all.json  test.json   trainval.json
output_json.json      train.json  val.json


In [3]:
pwd

'/root/workspace/mmyolo'

In [5]:
# dataset_root is the path , where images and the annotation file consists 
dataset_root=input('Enter the dataset root path')

# annotation_path , where there is a single file annotation in the coco (json) format 
annotation_path=input('Enter the annotation path ,(json file)')

# consists of images for training
train_percentage=input('Enter the percentage to divide the train dataset. Eg:0.8')
test_percentage= input('Enter the percentage to divide the test dataset. Eg:0.1')
val_percentage= input('Enter the percentage to divide the validation dataset. Eg:0.1')

# enter the epochs , upto how much epochs the models needs to train 
MAX_EPOCHS = input('Enter the MAX_EPOCHS ')

# BATCH_SIZE means  "The number of training examples utilized in one iteration".
BATCH_SIZE = input('Enter the BATCH_SIZE')

Enter the dataset root path /root/workspace/mmyolo/data/cat
Enter the annotation path ,(json file) /root/workspace/mmyolo/data/cat/annotations/annotations_all.json
Enter the percentage to divide the train dataset. Eg:0.8 0.8
Enter the percentage to divide the test dataset. Eg:0.1 0.1
Enter the percentage to divide the validation dataset. Eg:0.1 0.1
Enter the MAX_EPOCHS  100
Enter the BATCH_SIZE 4


# select the model , need to do testing and quantization

In [6]:
import sys

# Dictionary of model types and their corresponding options
model_types = {
    "YOLOx": ["yolox_tiny", "yolox_s", "yolox_m", "yolox_l", "yolox_x"],
    "RTMDET": ["rtmdet_tiny", "rtmdet_s", "rtmdet_l", "rtmdet_m", "rtmdet_x"],
    "YOLOv7": ["yolov7_tiny", "yolov7_l", "yolov7_x", "yolov7_w", "yolov7_e"],
}

# Batch size options
batch_size_options = [1, 2, 4, 8, 16]

# Variable to store user's choice
selected_model_type = None
selected_model_variant = None
test_option = None
path = None
quantize_option = None
quantize_bits = None
batch_size = None
AUGMENTATIONS = {
    "noise": False,
    "denoise": False,
    "contrast": False,
    "brightness": False,
    "rotation": False
}
# Print model types
print("Please select a model type:")
for i, model_type in enumerate(model_types):
    print(f"{i}. {model_type}")

# Get input for model type
while True:
    model_type_choice = input("Enter choice (0-{}): ".format(len(model_types) - 1))
    try:
        model_type_choice = int(model_type_choice)
        if 0 <= model_type_choice < len(model_types):
            selected_model_type = list(model_types.keys())[model_type_choice]
            break
    except ValueError:
        pass

    print("Invalid choice, please try again")

# Print model variants for the selected model type
print(f"Please select a {selected_model_type} model variant:")
for i, model_variant in enumerate(model_types[selected_model_type]):
    print(f"{i}. {model_variant}")

# Get input for model variant
while True:
    model_variant_choice = input("Enter choice (0-{}): ".format(len(model_types[selected_model_type]) - 1))
    try:
        model_variant_choice = int(model_variant_choice)
        if 0 <= model_variant_choice < len(model_types[selected_model_type]):
            selected_model_variant = model_types[selected_model_type][model_variant_choice]
            break
    except ValueError:
        pass

    print("Invalid choice, please try again")

print(f"You selected: {selected_model_variant}")
# Ask if the user wants to train the model with data augmentation
while True:
    train_with_augmentation = input("Do you want to train the model with data augmentation? (0 for no, 1 for yes): ")
    if train_with_augmentation in ["0", "1"]:
        break
    else:
        print("Invalid choice, please enter '0' or '1'.")
if train_with_augmentation == "1":
    print("Please specify whether you want to enable or disable each augmentation option (0 for False, 1 for True):")
    for augmentation, enabled in AUGMENTATIONS.items():
        while True:
            choice = input(f"Do you want to enable {augmentation.capitalize()}? (0 for False, 1 for True): ")
            if choice in ["0", "1"]:
                AUGMENTATIONS[augmentation] = choice == "1"
                break
            else:
                print("Invalid choice, please enter '0' or '1'.")
# Ask if the user wants to test
while True:
    test_option = input("Do you need to test? (0 for no, 1 for yes): ")
    if test_option in ["0", "1"]:
        break
    else:
        print("Invalid choice, please enter '0' or '1'.")

# If testing is required, ask for image or video and corresponding path
if test_option == "1":
    while True:
        test_type = input("Do you want to test on an image or a video? (0 for image, 1 for video): ")
        if test_type == "0":
            path = ''  # You can set a default value or leave it empty for images
            break
        elif test_type == "1":
            video_path = input("Enter the video path: ")
            output_path = input("Enter the path to save the video: ")

            break
        else:
            print("Invalid choice, please enter '0' or '1'.")

# Now 'path' will be either an empty string for images or the user-provided video path for videos

# Ask if the user wants to quantize the model
while True:
    quantize_option = input("Do you want to quantize the model? (0 for no, 1 for yes): ")
    if quantize_option in ["0", "1"]:
        break
    else:
        print("Invalid choice, please enter '0' or '1'.")

# If quantizing is required, ask for the quantization bit precision
if quantize_option == "1":
    while True:
        quantize_bits = input("Choose quantization precision (0 for fp16, 1 for fp32): ")
        if quantize_bits in ["0", "1"]:
            if quantize_bits == "0" or quantize_bits == "1":
                while True:
                    print("Choose batch size:")
                    for i, size in enumerate(batch_size_options):
                        print(f"{i}. {size}")

                    batch_size_choice = input("Enter choice (0-{}): ".format(len(batch_size_options) - 1))
                    try:
                        batch_size_choice = int(batch_size_choice)
                        if 0 <= batch_size_choice < len(batch_size_options):
                            batch_size = batch_size_options[batch_size_choice]
                            break
                    except ValueError:
                        pass

                    print("Invalid choice, please try again.")

            break
        else:
            print("Invalid choice, please enter '0' or '1'.")

print(f"You selected: {selected_model_variant}")
print(f"Testing option: {'yes' if test_option == '1' else 'no'}")
if test_option == "1":
    print(f"Testing on: {'image' if test_type == '0' else 'video'} at path: {path}")
print(f"Quantize option: {'yes' if quantize_option == '1' else 'no'}")
if quantize_option == "1":
    print(f"Quantization precision: {'fp16' if quantize_bits == '0' else 'fp32'}")
    print(f"Batch size: {batch_size}")


Please select a model type:
0. YOLOx
1. RTMDET
2. YOLOv7


Enter choice (0-2):  1


Please select a RTMDET model variant:
0. rtmdet_tiny
1. rtmdet_s
2. rtmdet_l
3. rtmdet_m
4. rtmdet_x


Enter choice (0-4):  3


You selected: rtmdet_m


...

Please specify whether you want to enable or disable each augmentation option (0 for False, 1 for True):


Do you want to enable Noise? (0 for False, 1 for True):  1
Do you want to enable Denoise? (0 for False, 1 for True):  0
Do you want to enable Contrast? (0 for False, 1 for True):  1
Do you want to enable Brightness? (0 for False, 1 for True):  1
Do you want to enable Rotation? (0 for False, 1 for True):  1
Do you need to test? (0 for no, 1 for yes):  1
Do you want to test on an image or a video? (0 for image, 1 for video):  0
Do you want to quantize the model? (0 for no, 1 for yes):  1
Choose quantization precision (0 for fp16, 1 for fp32):  1


Choose batch size:
0. 1
1. 2
2. 4
3. 8
4. 16


Enter choice (0-4):  3


You selected: rtmdet_m
Testing option: yes
Testing on: image at path: 
Quantize option: yes
Quantization precision: fp32
Batch size: 8


In [7]:
import os
parent_directory = os.path.dirname(annotation_path)
parent_directory

'/root/workspace/mmyolo/data/cat/annotations'

In [8]:
AUGMENTATIONS

{'noise': True,
 'denoise': False,
 'contrast': True,
 'brightness': True,
 'rotation': True}

In [9]:
# annotation_path="/root/workspace/mmyolo/data/cat/annotations/annotations_all.json"

In [10]:
train_with_augmentation

'1'

In [11]:
dataset_root

'/root/workspace/mmyolo/data/cat'

In [12]:
import os
import json
import random

def extract_random_middle_part(annotation_file_path, dataset_root):
    with open(annotation_file_path, 'r') as f:
        annotation_data = json.load(f)

    # Get a random image from the list
    random_image = random.choice(annotation_data["images"])

    file_path = random_image["file_name"]
    relative_path = os.path.relpath(file_path, dataset_root)
    middle_part = os.path.dirname(relative_path)

    return middle_part

# Example usage
# annotation_file_path = "/path/to/your/annotation_file.json"
# dataset_root = "/home/viso/sample_dataset"

random_middle_part = extract_random_middle_part(annotation_path, dataset_root)

# Print the extracted middle part
print(random_middle_part)


../..


In [13]:
random_middle_part='images'

In [14]:
random_middle_part

'images'

In [15]:
pwd

'/root/workspace/mmyolo'

# Generating metainfo for the config based on the annotation file 

In [16]:
import json

with open(annotation_path, 'r') as f:
    data = json.load(f)

classes = [c['name'] for c in data['categories']]
NUM_CLASSES = len(classes)
palette = []

for i, c in enumerate(classes):
    r = (i * 50) % 255
    g = (i * 100) % 255
    b = (i * 150) % 255
    palette.append((r, g, b))

metainfo = {
    'classes': classes,
    'palette': palette
}
class_names=metainfo['classes']
print("Class Names:", class_names)
print("Number of Classes:", NUM_CLASSES)
print("Metainfo:", metainfo)


Class Names: ['cat']
Number of Classes: 1
Metainfo: {'classes': ['cat'], 'palette': [(0, 0, 0)]}


In [17]:
NUM_CLASSES

1

# Data Augmentation 

# converting the annotations from json to txt  

In [18]:
import json
import shutil
# Load COCO annotations 
def convert_coco_to_yolo_old(annotation_path,dataset_root):
    print(".......... in convert_coco_to_yolo ..........")
    print(annotation_path)
    # Ensure the "txt_files" directory exists or create it
    txt_files_dir = os.path.join(dataset_root, 'txt_files')
    print(txt_files_dir)
    if not os.path.exists(txt_files_dir):
        os.makedirs(txt_files_dir)

    with open(annotation_path) as f:
        coco = json.load(f)


    # Create lists to store image info and annotations
    image_ids = []
    image_info = []
    annotations = []
    print("image_ids",image_ids)
    print("image_info",image_info)
    print("annotations",annotations)
    # Get image ids and image info 
    for img in coco['images']:
        image_ids.append(img['id'])
        image_info.append({'file_name': img['file_name'],
                           'width': img['width'],
                           'height': img['height']})
        # print("image id are",image_ids)
    # print(".........................")
    # Get annotations 
    for ann in coco['annotations']:
        bbox = ann['bbox']
        x, y, width, height = bbox
        image_id = ann['image_id']
        category_id = ann['category_id']
        # print("image id 'ssssssssss",image_id)
        # Convert to relative cooridnates 
        x_rel = x/image_info[image_id-1]['width'] 
        y_rel = y/image_info[image_id-1]['height']
        width_rel = width/image_info[image_id-1]['width']
        height_rel = height/image_info[image_id-1]['height']

        annotation = (category_id, x_rel, y_rel, width_rel, height_rel)
        annotations.append({'image_id': image_id, 'annotation': annotation})

    # Convert to YOLO format
    yolo_annotations = []
    for annotation in annotations:
        image_id = annotation['image_id']
        # print("image_id",image_id)
        category_id, x, y, width, height = annotation['annotation']

        # Calculate center x, y 
        x_center = x + width/2
        y_center = y + height/2

        yolo_annotation = f"{category_id} {x_center} {y_center} {width} {height}\n"
        yolo_annotations.append({'image_id': image_id, 'annotation': yolo_annotation})

    # Group annotations by image    
    yolo_files = {}
    for annotation in yolo_annotations:
        print(yolo_annotations)
        image_id = annotation['image_id']
        yolo_annotation = annotation['annotation']
        if image_id not in yolo_files:
            yolo_files[image_id] = [yolo_annotation]
        else:
            yolo_files[image_id].append(yolo_annotation)


    # Save annotation files
        for image_id, annotations in yolo_files.items(): 
            file_name = image_info[image_id-1]['file_name']
            # print('.1.',file_name)
            file_name = file_name.split('.')[0] + '.txt'
            # print('.2.',file_name)
            base_name = os.path.splitext(os.path.basename(file_name))[0]  # Get the base name without extension

            # Construct the destination .txt file name
            txt_file_name = base_name + '.txt'
            file_path = os.path.join(txt_files_dir, txt_file_name)

            #print(f"Saving to: {file_path}")

            with open(file_path, 'w') as f:
            # with open(txt_files_dir, 'w') as f:
                for annotation in annotations:
                    f.write(annotation)

            # with open(file_path, 'w') as f:
            #     for annotation in annotations:
            #         f.write(annotation + '\n')
            #         # destination_path = os.path.join(txt_files_dir, file_name)
            #         shutil.move(file_path, txt_files_dir)



    # Save image list file
    # with open('train.txt', 'w') as f:
    #     for image_id in image_ids:
    #         file_name = image_info[image_id-1]['file_name']
    #         f.write(file_name + '\n')
    
#convert_coco_to_yolo(annotation_path, dataset_root)


In [19]:
import json
import os
import shutil

def convert_coco_to_yolo(annotation_path, dataset_root):
    print(".......... in convert_coco_to_yolo ..........")
    print(annotation_path)

    # Ensure the "txt_files" directory exists or create it
    txt_files_dir = os.path.join(dataset_root, 'txt_files')
    print(txt_files_dir)
    if not os.path.exists(txt_files_dir):
        os.makedirs(txt_files_dir)

    with open(annotation_path) as f:
        coco = json.load(f)

    # Create lists to store image info and annotations
    image_info = {}
    annotations = []

    # Get image info
    for img in coco['images']:
        image_info[img['id']] = {'file_name': img['file_name'],
                                  'width': img['width'],
                                  'height': img['height']}

    # Get annotations 
    for ann in coco['annotations']:
        bbox = ann['bbox']
        x, y, width, height = bbox
        image_id = ann['image_id']
        if image_id not in image_info:
            print(f"Warning: Image ID {image_id} not found in image info")
            continue

        category_id = ann['category_id']
        
        # Convert to relative coordinates 
        x_rel = x / image_info[image_id]['width']
        y_rel = y / image_info[image_id]['height']
        width_rel = width / image_info[image_id]['width']
        height_rel = height / image_info[image_id]['height']

        annotation = (category_id, x_rel, y_rel, width_rel, height_rel)
        annotations.append({'image_id': image_id, 'annotation': annotation})

    # Convert to YOLO format
    yolo_annotations = []
    for annotation in annotations:
        image_id = annotation['image_id']
        category_id, x, y, width, height = annotation['annotation']

        # Calculate center x, y 
        x_center = x + width / 2
        y_center = y + height / 2

        yolo_annotation = f"{category_id} {x_center} {y_center} {width} {height}\n"
        yolo_annotations.append({'image_id': image_id, 'annotation': yolo_annotation})

    # Group annotations by image    
    yolo_files = {}
    for annotation in yolo_annotations:
        image_id = annotation['image_id']
        yolo_annotation = annotation['annotation']
        if image_id not in yolo_files:
            yolo_files[image_id] = [yolo_annotation]
        else:
            yolo_files[image_id].append(yolo_annotation)

    # Save annotation files
    for image_id, annotations in yolo_files.items(): 
        if image_id not in image_info:
            print(f"Error: Image ID {image_id} not found in image info")
            continue
            
        file_name = image_info[image_id]['file_name']
        base_name = os.path.splitext(os.path.basename(file_name))[0]  # Get the base name without extension
        txt_file_name = base_name + '.txt'
        file_path = os.path.join(txt_files_dir, txt_file_name)

        with open(file_path, 'w') as f:
            for annotation in annotations:
                f.write(annotation)

    # Save image list file
    # with open('train.txt', 'w') as f:
    #     for image_id in image_ids:
    #         file_name = image_info[image_id-1]['file_name']
    #         f.write(file_name + '\n')

# Now let's call the function
# annotation_path = "/home/viso/sample_data/annotations/total_annotations.json"
# dataset_root = "/home/viso/sample_data/augmented_images"
# convert_coco_to_yolo(annotation_path, dataset_root)


# creating the config file for the data augmentation 

In [20]:
def edit_and_save_config(random_middle_part, dataset_root,AUGMENTATIONS):
    print("............edit_and_save_config..............")
    # %cd /home/viso/datasets/enhance_script
    import json
    new_config_path = '/root/workspace/mmyolo/quantize/dataset_enhance_config.json'
    original_config_path = '/root/workspace/mmyolo/quantize/dataset_enhance_config-dummy.json'
    # Load the original config
    with open(original_config_path) as f:
        config = json.load(f)

    # Modify the content as needed
    config["original_image_folder"] = f'{dataset_root}/{random_middle_part}'
    config["original_label_folder"] = f'{dataset_root}/txt_files'
    config["enhance_image_folder"] = f'{dataset_root}/train'
    config["enhance_label_folder"] = f'{dataset_root}/train'
    config["augmentations"] = {
        "noise": AUGMENTATIONS['noise'],
        "denoise": AUGMENTATIONS['denoise'],
        "contrast": AUGMENTATIONS['contrast'],
        "brightness": AUGMENTATIONS['brightness'],
        "rotation": AUGMENTATIONS['rotation']
    }
    # Save the modified config to a new file
    with open(new_config_path, 'w') as f:
        json.dump(config, f, indent=2)

# Example usage:

# random_middle_part = 'images'
# dataset_root = '/home/viso/sample_dataset'  # Provide the actual path
# edit_and_save_config(original_config_path, new_config_path, random_middle_part, dataset_root)


# Getting all the image_paths in the txt file 

In [21]:
import os

def get_image_paths(folder_path, extensions=[".jpg", ".png"]):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths

def write_paths_to_file(image_paths, output_file=f"{dataset_root}/train.txt"):
    with open(output_file, "w") as file:
        for path in image_paths:
            file.write(path + "\n")

# if __name__ == "__main__":
# folder_path = "/home/viso/sample_data/augmented_imagges"  # Replace with the actual path to your folder
    # image_paths = get_image_paths(folder_path)
    # write_paths_to_file(image_paths)


In [22]:
import os

def update_config_with_class_names(class_names):
    dummy_main = '/root/workspace/mmyolo/quantize/dummy_main.py'
    main='/root/workspace/mmyolo/quantize/main.py'
    with open(dummy_main, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("classes =", f"classes = {class_names}")

    # Write the updated content back to the configuration file
    with open(main, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

# Example usage:
# number_of_classes = 10  # Replace with the actual number of classes


In [23]:
import shutil
if train_with_augmentation=='1':
    # %cd /home/viso/notebooks
    # %run ./aug.ipynb
    convert_coco_to_yolo(annotation_path, dataset_root)
    edit_and_save_config(random_middle_part, dataset_root,AUGMENTATIONS)
    %cd /root/workspace/mmyolo/quantize
    !python run_dataset_enhance.py
    # %cd /home/viso/github/Yolo-to-COCO-format-converter
    update_config_with_class_names(class_names)
    !pip install imagesize
    !python main.py --path {dataset_root} --output output_json.json
    %pwd
    shutil.copy('/root/workspace/mmyolo/quantize/output/output_json.json',parent_directory)
    %cd {dataset_root}
    folder_path=f'{dataset_root}/train'
    image_paths = get_image_paths(folder_path)
    write_paths_to_file(image_paths)
    with open('object.names', 'w') as f:
        for class_name in class_names:
            f.write(class_name + '\n')

.......... in convert_coco_to_yolo ..........
/root/workspace/mmyolo/data/cat/annotations/annotations_all.json
/root/workspace/mmyolo/data/cat/txt_files
............edit_and_save_config..............
/root/workspace/mmyolo/quantize
<class 'dict'>
Augmentations dictionary: {'noise': True, 'denoise': False, 'contrast': True, 'brightness': True, 'rotation': True}
Type of augmentations: <class 'dict'>
Value of augmentations['noise']: True
en_noise True
en_contrast True
en_bright True
Augmentations dictionary: {'noise': True, 'denoise': False, 'contrast': True, 'brightness': True, 'rotation': True}
Type of augmentations: <class 'dict'>
Value of augmentations['noise']: True
en_noise True
en_contrast True
en_bright True
Augmentations dictionary: {'noise': True, 'denoise': False, 'contrast': True, 'brightness': True, 'rotation': True}
Type of augmentations: <class 'dict'>
Value of augmentations['noise']: True
en_noise True
en_contrast True
en_bright True
Augmentations dictionary: {'noise': Tru

In [24]:
if train_with_augmentation=='1':
    annotation_path = f'{parent_directory}/output_json.json'
    print(annotation_path)
    random_middle_part = 'train'

/root/workspace/mmyolo/data/cat/annotations/output_json.json


In [25]:
print(type(AUGMENTATIONS))

<class 'dict'>


In [26]:
AUGMENTATIONS['noise']

True

# split the annotation file into train , test and validation 

### If there is single annotation file , that can divide into train , test and validation (coco format)

In [27]:
pwd

'/root/workspace/mmyolo/data/cat'

In [28]:
!pip install sklearn
!pip install funcy
!pip install argparse
!pip install scikit-learn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [29]:
!python cocosplit_train_test_valid.py \
    --annotations "{annotation_path}" \
    --train_ratio {train_percentage} \
    --valid_ratio {val_percentage} \
    --test_ratio {test_percentage} \
    --trainJson_name train.json \
    --validJson_name val.json \
    --testJson_name test.json


python: can't open file 'cocosplit_train_test_valid.py': [Errno 2] No such file or directory


In [30]:
pwd

'/root/workspace/mmyolo/data/cat'

In [31]:
import os , shutil
# dataset_root = '/home/viso/datasets/viso_datasets/construction-demo'

# parent_directory = os.path.dirname(annotation_path)
shutil.copy('/root/workspace/mmyolo/test.json',parent_directory)
shutil.copy('/root/workspace/mmyolo/val.json',parent_directory)
shutil.copy('/root/workspace/mmyolo/train.json',parent_directory)

train_json = os.path.join(parent_directory, 'train.json')
val_json = os.path.join(parent_directory, 'val.json')
test_json = os.path.join(parent_directory, 'test.json')

# Remove the dataset_root prefix using os.path.relpath
train_json_relpath = os.path.relpath(train_json, dataset_root)
val_json_relpath = os.path.relpath(val_json, dataset_root)
test_json_relpath = os.path.relpath(test_json, dataset_root)

print(train_json_relpath, val_json_relpath, test_json_relpath)
print(parent_directory)

annotations/train.json annotations/val.json annotations/test.json
/root/workspace/mmyolo/data/cat/annotations


# Yolox configuration setup 

In [32]:
# Check if the selected model type is 'YOLOx'
if selected_model_type == 'YOLOx':
    # Provide the path to the user for modification
    dummy_config = '/root/workspace/mmyolo/configs/yolox/custom_config.py'
    custom_config_file_path_base = '/root/workspace/mmyolo/configs/yolox/yolox_s_fast_8xb8-300e_coco.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("data_root =", f"data_root = '{dataset_root}/'")
    cfg_content = cfg_content.replace("train_data_prefix =", f"train_data_prefix = '{random_middle_part}'")
    cfg_content = cfg_content.replace("val_data_prefix =", f"val_data_prefix = '{random_middle_part}'")
    cfg_content = cfg_content.replace("train_ann_file =", f"train_ann_file = '{train_json_relpath}'")
    cfg_content = cfg_content.replace("val_ann_file =", f"val_ann_file = '{val_json_relpath}'")
    cfg_content = cfg_content.replace("num_classes =", f"num_classes = {NUM_CLASSES}")
    cfg_content = cfg_content.replace("meta_info =", f"meta_info = {metainfo}")
    cfg_content = cfg_content.replace("max_epochs =", f"max_epochs = {MAX_EPOCHS}")
    cfg_content = cfg_content.replace("train_batch_size_per_gpu =", f"train_batch_size_per_gpu = {BATCH_SIZE}")

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")
    print("dummy_config:", dummy_config)
    print("custom_config_file_path_base:", custom_config_file_path_base)


# Yolov7 configuration setup 

In [34]:
if selected_model_type == 'YOLOv7':
    dummy_config = '/root/workspace/mmyolo/configs/yolov7/custom_config.py'
    custom_config_file_path_base = '/root/workspace/mmyolo/configs/yolov7/yolov7_l_syncbn_fast_8x16b-300e_coco.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("data_root =", f"data_root = '{dataset_root}/'")
    cfg_content = cfg_content.replace("train_data_prefix =", f"train_data_prefix = '{random_middle_part}'")
    cfg_content = cfg_content.replace("val_data_prefix =", f"val_data_prefix = '{random_middle_part}'")
    cfg_content = cfg_content.replace("train_ann_file =", f"train_ann_file = '{train_json_relpath}'")
    cfg_content = cfg_content.replace("val_ann_file =", f"val_ann_file = '{val_json_relpath}'")
    cfg_content = cfg_content.replace("num_classes =", f"num_classes = {NUM_CLASSES}")
    cfg_content = cfg_content.replace("meta_info =", f"meta_info = {metainfo}")
    cfg_content = cfg_content.replace("max_epochs =", f"max_epochs = {MAX_EPOCHS}")
    cfg_content = cfg_content.replace("train_batch_size_per_gpu =", f"train_batch_size_per_gpu = {BATCH_SIZE}")

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")

# Rtmdet configuration setup

In [55]:
if selected_model_type == 'RTMDET':
    dummy_config = '/root/workspace/mmyolo/configs/rtmdet/custom_config.py'
    custom_config_file_path_base = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_l_syncbn_fast_8xb32-300e_coco.py'  # Set your desired path

    # Load the YOLOx config file
    with open(dummy_config, 'r') as f:
        cfg_content = f.read()

    # Replace specific lines with new values
    cfg_content = cfg_content.replace("data_root =", f"data_root = '{dataset_root}/'")
    cfg_content = cfg_content.replace("train_data_prefix =", f"train_data_prefix = '{random_middle_part}'")
    cfg_content = cfg_content.replace("val_data_prefix =", f"val_data_prefix = '{random_middle_part}'")
    cfg_content = cfg_content.replace("train_ann_file =", f"train_ann_file = '{train_json_relpath}'")
    cfg_content = cfg_content.replace("val_ann_file =", f"val_ann_file = '{val_json_relpath}'")
    cfg_content = cfg_content.replace("num_classes =", f"num_classes = {NUM_CLASSES}")
    cfg_content = cfg_content.replace("meta_info =", f"meta_info = {metainfo}")
    cfg_content = cfg_content.replace("max_rrandom_middle_partandom_middle_partepochs =", f"max_epochs = {MAX_EPOCHS}")
    cfg_content = cfg_content.replace("train_batch_size_per_gpu =", f"train_batch_size_per_gpu = {BATCH_SIZE}")

    # Save the modified content to the new file
    with open(custom_config_file_path_base, 'w') as custom_cfg_file:
        custom_cfg_file.write(cfg_content)

    print(f"Custom config file has been created at {custom_config_file_path_base}.")
    print("dummy_config:", dummy_config)


In [36]:
if selected_model_variant=='yolox_tiny':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_tiny_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_s':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_s_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_m':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_m_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_l':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_l_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolox_x':
    config_file_path = '/root/workspace/mmyolo/configs/yolox/yolox_x_fast_8xb8-300e_coco.py'
elif selected_model_variant == 'yolov7_tiny':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_tiny_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_l':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_l_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_x':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_x_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_w':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_w-p6_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'yolov7_e':
    config_file_path = '/root/workspace/mmyolo/configs/yolov7/yolov7_e_syncbn_fast_8x16b-300e_coco.py'
elif selected_model_variant == 'rtmdet_tiny':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_tiny_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_s':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_s_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_l':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_l_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_m':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_m_syncbn_fast_8xb32-300e_coco.py'
elif selected_model_variant == 'rtmdet_x':
    config_file_path = '/root/workspace/mmyolo/configs/rtmdet/rtmdet_x_syncbn_fast_8xb32-300e_coco.py'

In [37]:
custom_config_file_path=config_file_path.split('/')[-1]

# train the data with the selected model 

In [38]:
%cd /root/workspace/mmyolo/

/root/workspace/mmyolo


In [39]:
selected_model_type=selected_model_type.lower()
print(selected_model_type)
custom_config_file_path

rtmdet


'rtmdet_m_syncbn_fast_8xb32-300e_coco.py'

In [54]:
MAX_EPOCHS

'10'

In [57]:
import subprocess

# Assuming config_file_path is something like "yolox/config_file.py"
# config_file_path = config_file_path.split('/')[-1]
command_line = f"python tools/train.py configs/{selected_model_type}/{custom_config_file_path}"

# Run the command
result = subprocess.run(command_line, shell=True)

# Check the result
if result.returncode == 0:
    print("Command executed successfully")
else:
    print(f"Error executing the command. Return code: {result.returncode}")
    print(f"Output: {result.stdout}")
    print(f"Error: {result.stderr}")


05/01 16:52:07 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "log_processor" registry tree. As a workaround, the current "log_processor" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmyolo" is a correct scope, or whether the registry is initialized.
05/01 16:52:07 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.19 (default, Mar 20 2024, 19:58:24) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1262772697
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.10.0
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 2

/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


05/01 16:52:27 - mmengine - INFO - Exp name: rtmdet_m_syncbn_fast_8xb32-300e_coco_20240501_165207
05/01 16:52:27 - mmengine - INFO - Epoch(train)  [1][4/4]  base_lr: 1.2052e-05 lr: 1.2052e-05  eta: 0:01:53  time: 3.1572  data_time: 0.3383  memory: 12154  loss: 0.0600  loss_cls: 0.0320  loss_bbox: 0.0280


/opt/conda/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


05/01 16:52:34 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Rec

# testing with the trained model with image

In [58]:
import os
# Remove the last .py extension
custom_config_file_path_without_extension = os.path.splitext(custom_config_file_path)[0]

print(custom_config_file_path_without_extension)


rtmdet_m_syncbn_fast_8xb32-300e_coco


In [59]:
test_images='images'

In [60]:
metainfo

{'classes': ['cat'], 'palette': [(0, 0, 0)]}

In [61]:
if test_option == '1' and test_type == '0':
    %cd /root/workspace/mmyolo
    %run ./test.ipynb
    testing_image(MAX_EPOCHS, custom_config_file_path_base,test_images,test_json_relpath,dummy_config,metainfo,BATCH_SIZE,NUM_CLASSES,dataset_root)   
    %cd /root/workspace/mmyolo
    import subprocess
    command_line = f"python tools/test.py \
        configs/{selected_model_type}/{custom_config_file_path} \
        work_dirs/{custom_config_file_path_without_extension}/epoch_{MAX_EPOCHS}.pth \
        --work-dir wk"

    # Run the command
    result = subprocess.run(command_line, shell=True)

    # Check the result
    if result.returncode == 0:
        print("Command executed successfully")
    else:
        print(f"Error executing the command. Return code: {result.returncode}")
        print(f"Output: {result.stdout}")
        print(f"Error: {result.stderr}")

/root/workspace/mmyolo
dummy_config: /root/workspace/mmyolo/configs/rtmdet/custom_config.py
custom_config_file_path_base: /root/workspace/mmyolo/configs/rtmdet/rtmdet_l_syncbn_fast_8xb32-300e_coco.py
/root/workspace/mmyolo
05/01 16:53:44 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "log_processor" registry tree. As a workaround, the current "log_processor" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmyolo" is a correct scope, or whether the registry is initialized.
05/01 16:53:44 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.19 (default, Mar 20 2024, 19:58:24) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1162776163
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    G

/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


05/01 16:53:59 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Rec

/opt/conda/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Command executed successfully


# testing with the trained model with video

In [62]:
if test_option == '1' and test_type == '1':
    print(".....testing on video......")
    %cd /root/workspace/mmyolo
    %run ./test.ipynb
    testing_video(MAX_EPOCHS, custom_config_file_path_base,selected_model_type,video_path,output_path,custom_config_file_path)

# Quantize the model (convert the pytorch model into tensorrt model )

In [ ]:
if quantize_option == '1':
    %cd /root/workspace/mmyolo
    %run ./quantize.ipynb
    quantize(quantize_option, quantize_bits, batch_size_choice)
    import subprocess
    %cd /root/workspace/
    command_line = f"python mmdeploy/tools/deploy.py \
    /root/workspace/mmyolo/configs/deploy/detection_tensorrt_dynamic-192x192-960x960.py \
    {config_file_path} \
    mmyolo/work_dirs/{custom_config_file_path_without_extension}/epoch_{MAX_EPOCHS}.pth \
    mmyolo/demo/demo.jpg \
    --work-dir mmdeploy_model/{custom_config_file_path}_tensorrt \
    --device cuda \
    --dump-info"

    # Run the command
    result = subprocess.run(command_line, shell=True)

    # Check the result
    if result.returncode == 0:
        print("Command executed successfully")
    else:
        print(f"Error executing the command. Return code: {result.returncode}")
        print(f"Output: {result.stdout}")
        print(f"Error: {result.stderr}")

/root/workspace/mmyolo
........in quantize...........
/root/workspace
05/01 16:54:05 - mmengine - WARNING - Import mmdeploy.codebase.mmyolo.deploy failedPlease check whether the module is the custom module.No module named 'mmdeploy.codebase.mmyolo'
05/01 16:54:05 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmyolo" is a correct scope, or whether the registry is initialized.
05/01 16:54:07 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
05/01 16:54:09 - mmengine - WARNING - Import mmdeploy.codebase.mmyolo.deploy failedPlease check whether the module is the custom module.No module named 'mmdeploy.codebase.mmyolo'
05/01 16:54:09 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "Codebases" re

05/01 16:54:25 - mmengine - INFO - Execute onnx optimize passes.
05/01 16:54:25 - mmengine - INFO - Finish pipeline mmdeploy.apis.pytorch2onnx.torch2onnx


/root/workspace/mmdeploy/mmdeploy/core/optimizers/function_marker.py:160: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  ys_shape = tuple(int(s) for s in ys.shape)
/opt/conda/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/root/workspace/mmyolo/mmyolo/models/task_modules/coders/distance_point_bbox_coder.py:45: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treat

05/01 16:54:28 - mmengine - INFO - Start pipeline mmdeploy.apis.utils.utils.to_backend in subprocess
05/01 16:54:28 - mmengine - INFO - Successfully loaded tensorrt plugins from /root/workspace/mmdeploy/mmdeploy/lib/libmmdeploy_tensorrt_ops.so
[05/01/2024-16:54:29] [TRT] [I] [MemUsageChange] Init CUDA: CPU +605, GPU +0, now: CPU 690, GPU 256 (MiB)
[05/01/2024-16:54:30] [TRT] [I] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 690 MiB, GPU 256 MiB
[05/01/2024-16:54:30] [TRT] [I] [MemUsageSnapshot] End constructing builder kernel library: CPU 849 MiB, GPU 290 MiB
[05/01/2024-16:54:30] [TRT] [I] ----------------------------------------------------------------
[05/01/2024-16:54:30] [TRT] [I] Input filename:   mmdeploy_model/rtmdet_m_syncbn_fast_8xb32-300e_coco.py_tensorrt/end2end.onnx
[05/01/2024-16:54:30] [TRT] [I] ONNX IR version:  0.0.7
[05/01/2024-16:54:30] [TRT] [I] Opset version:    11
[05/01/2024-16:54:30] [TRT] [I] Producer name:    pytorch
[05/01/2024-16:54:30] [